In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.04
schedule = [75, 175, 250]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


# Loss

In [10]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [11]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [12]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [13]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 300] LR: 0.040000
1/643 Data:1.907 | Batch:8.008 | Total:0:00:08 | ETA:1:25:42 | Loss:0.7159047722816467 | top1:45.5
11/643 Data:0.001 | Batch:0.719 | Total:0:00:15 | ETA:0:15:37 | Loss:0.705941146070307 | top1:49.90909194946289
21/643 Data:0.001 | Batch:0.830 | Total:0:00:23 | ETA:0:08:39 | Loss:0.8370623333113534 | top1:49.42856979370117
31/643 Data:0.001 | Batch:0.658 | Total:0:00:31 | ETA:0:07:28 | Loss:0.8107202764480345 | top1:49.91935729980469
41/643 Data:0.001 | Batch:0.838 | Total:0:00:39 | ETA:0:07:54 | Loss:0.798310266762245 | top1:49.92683029174805
51/643 Data:0.002 | Batch:0.697 | Total:0:00:47 | ETA:0:08:09 | Loss:0.7897500337338915 | top1:49.892154693603516
61/643 Data:0.010 | Batch:0.752 | Total:0:00:54 | ETA:0:07:13 | Loss:0.7794680458600404 | top1:49.8278694152832
71/643 Data:0.000 | Batch:0.688 | Total:0:01:01 | ETA:0:06:46 | Loss:0.7701235724167085 | top1:49.71126937866211
81/643 Data:0.001 | Batch:0.708 | Total:0:01:08 | ETA:0:06:39 | Loss:0.7638516632

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.001 | Batch:1.209 | Total:0:00:35 | ETA:0:00:00 | Loss:0.20523508607969848 | top1:96.06854248046875

Epoch: [2 | 300] LR: 0.068000
1/643 Data:2.844 | Batch:3.807 | Total:0:00:03 | ETA:0:40:46 | Loss:0.4230310022830963 | top1:81.5
11/643 Data:0.002 | Batch:0.704 | Total:0:00:09 | ETA:0:09:31 | Loss:0.4606305414980108 | top1:78.5
21/643 Data:0.011 | Batch:0.728 | Total:0:00:16 | ETA:0:07:02 | Loss:0.45552063697860357 | top1:78.73809814453125
31/643 Data:0.011 | Batch:0.752 | Total:0:00:23 | ETA:0:07:01 | Loss:0.4534732295620826 | top1:78.58065032958984
41/643 Data:0.005 | Batch:0.693 | Total:0:00:30 | ETA:0:07:08 | Loss:0.4533067163897724 | top1:78.70731353759766
51/643 Data:0.006 | Batch:0.894 | Total:0:00:37 | ETA:0:07:05 | Loss:0.4594135097428864 | top1:78.33333587646484
61/643 Data:0.006 | Batch:0.756 | Total:0:00:45 | ETA:0:07:51 | Loss:0.4566745362320884 | top1:78.54917907714844
71/643 Data:0.020 | Batch:0.784 | Total:0:00:53 | ETA:0:07:18 | Loss:0.45480430336065697 

51/643 Data:0.004 | Batch:0.957 | Total:0:00:42 | ETA:0:07:45 | Loss:0.4169195335285336 | top1:80.7450942993164
61/643 Data:0.013 | Batch:0.626 | Total:0:00:50 | ETA:0:07:57 | Loss:0.4172155930370581 | top1:80.7213134765625
71/643 Data:0.012 | Batch:0.829 | Total:0:00:58 | ETA:0:07:42 | Loss:0.4191730929092622 | top1:80.5281753540039
81/643 Data:0.001 | Batch:0.692 | Total:0:01:06 | ETA:0:07:16 | Loss:0.418488308235451 | top1:80.53703308105469
91/643 Data:0.011 | Batch:0.829 | Total:0:01:13 | ETA:0:07:01 | Loss:0.41890414992531577 | top1:80.489013671875
101/643 Data:0.009 | Batch:0.783 | Total:0:01:21 | ETA:0:06:37 | Loss:0.4196469544774235 | top1:80.38613891601562
111/643 Data:0.007 | Batch:0.724 | Total:0:01:29 | ETA:0:07:10 | Loss:0.42093378708169266 | top1:80.24324798583984
121/643 Data:0.002 | Batch:0.686 | Total:0:01:36 | ETA:0:06:17 | Loss:0.4199793848617018 | top1:80.32644653320312
131/643 Data:0.002 | Batch:0.647 | Total:0:01:43 | ETA:0:05:56 | Loss:0.42034811941721967 | top1:

111/643 Data:0.008 | Batch:0.740 | Total:0:01:26 | ETA:0:06:10 | Loss:0.4173905860196363 | top1:80.47297668457031
121/643 Data:0.005 | Batch:0.875 | Total:0:01:33 | ETA:0:06:29 | Loss:0.4173171906924445 | top1:80.43801879882812
131/643 Data:0.001 | Batch:0.886 | Total:0:01:42 | ETA:0:07:13 | Loss:0.4154964678615104 | top1:80.53053283691406
141/643 Data:0.001 | Batch:0.799 | Total:0:01:50 | ETA:0:06:50 | Loss:0.4167189841152083 | top1:80.40071105957031
151/643 Data:0.014 | Batch:0.793 | Total:0:01:58 | ETA:0:06:55 | Loss:0.4164962020536132 | top1:80.43376922607422
161/643 Data:0.011 | Batch:0.789 | Total:0:02:06 | ETA:0:06:11 | Loss:0.4156876502570158 | top1:80.52173614501953
171/643 Data:0.001 | Batch:0.636 | Total:0:02:13 | ETA:0:05:47 | Loss:0.4151882185922032 | top1:80.55847930908203
181/643 Data:0.006 | Batch:0.684 | Total:0:02:21 | ETA:0:05:35 | Loss:0.41459545904760203 | top1:80.60773468017578
191/643 Data:0.003 | Batch:0.773 | Total:0:02:28 | ETA:0:05:32 | Loss:0.413874995333986

...

601/643 Data:0.001 | Batch:0.731 | Total:0:06:58 | ETA:0:00:29 | Loss:0.02217594799347386 | top1:99.16638946533203
611/643 Data:0.003 | Batch:0.721 | Total:0:07:04 | ETA:0:00:23 | Loss:0.02215900622224801 | top1:99.16693878173828
621/643 Data:0.001 | Batch:0.672 | Total:0:07:12 | ETA:0:00:16 | Loss:0.022167008389376144 | top1:99.16586303710938
631/643 Data:0.000 | Batch:0.686 | Total:0:07:19 | ETA:0:00:09 | Loss:0.022021948798355187 | top1:99.17115020751953
641/643 Data:0.001 | Batch:0.702 | Total:0:07:25 | ETA:0:00:02 | Loss:0.022054514083550015 | top1:99.16770935058594
161/161 Data:0.002 | Batch:0.141 | Total:0:00:32 | ETA:0:00:00 | Loss:0.053587507714032895 | top1:98.5015640258789

Epoch: [151 | 300] LR: 0.017839
1/643 Data:1.627 | Batch:2.358 | Total:0:00:02 | ETA:0:25:15 | Loss:0.00990247167646885 | top1:99.5
11/643 Data:0.002 | Batch:0.702 | Total:0:00:09 | ETA:0:09:09 | Loss:0.020877713549204847 | top1:99.45454406738281
21/643 Data:0.002 | Batch:0.673 | Total:0:00:16 | ETA:0:07:

161/161 Data:0.000 | Batch:0.200 | Total:0:00:32 | ETA:0:00:00 | Loss:0.03151894472191267 | top1:99.0249252319336

Epoch: [152 | 300] LR: 0.017672
1/643 Data:3.965 | Batch:4.780 | Total:0:00:04 | ETA:0:51:09 | Loss:0.029981479048728943 | top1:98.5
11/643 Data:0.001 | Batch:0.794 | Total:0:00:12 | ETA:0:11:51 | Loss:0.022506459108130497 | top1:99.04545593261719
21/643 Data:0.001 | Batch:0.631 | Total:0:00:19 | ETA:0:07:25 | Loss:0.020135369595317615 | top1:99.14285278320312
31/643 Data:0.001 | Batch:0.701 | Total:0:00:26 | ETA:0:08:02 | Loss:0.02093088487163186 | top1:99.20967864990234
41/643 Data:0.006 | Batch:0.781 | Total:0:00:34 | ETA:0:07:25 | Loss:0.021630226861594652 | top1:99.25609588623047
51/643 Data:0.001 | Batch:0.675 | Total:0:00:41 | ETA:0:07:12 | Loss:0.020646562489370506 | top1:99.28430938720703
61/643 Data:0.001 | Batch:0.794 | Total:0:00:49 | ETA:0:07:14 | Loss:0.020239650748944918 | top1:99.29508209228516
71/643 Data:0.001 | Batch:0.692 | Total:0:00:56 | ETA:0:07:03 |

41/643 Data:0.003 | Batch:0.667 | Total:0:00:30 | ETA:0:06:54 | Loss:0.017563986432988468 | top1:99.41463470458984
51/643 Data:0.001 | Batch:0.734 | Total:0:00:37 | ETA:0:06:53 | Loss:0.017222011302068244 | top1:99.39215850830078
61/643 Data:0.001 | Batch:0.697 | Total:0:00:44 | ETA:0:06:49 | Loss:0.016965345490998665 | top1:99.36885070800781
71/643 Data:0.001 | Batch:0.639 | Total:0:00:51 | ETA:0:06:41 | Loss:0.01832527407890999 | top1:99.3239517211914
81/643 Data:0.001 | Batch:0.714 | Total:0:00:58 | ETA:0:06:27 | Loss:0.018274293979629874 | top1:99.32098388671875
91/643 Data:0.003 | Batch:0.686 | Total:0:01:05 | ETA:0:06:30 | Loss:0.018084264171033444 | top1:99.32418060302734
101/643 Data:0.002 | Batch:0.700 | Total:0:01:12 | ETA:0:06:14 | Loss:0.01792397066757157 | top1:99.34158325195312
111/643 Data:0.006 | Batch:0.737 | Total:0:01:19 | ETA:0:06:16 | Loss:0.017852957691863045 | top1:99.35135650634766
121/643 Data:0.001 | Batch:0.669 | Total:0:01:26 | ETA:0:06:07 | Loss:0.017561588

91/643 Data:0.002 | Batch:0.701 | Total:0:01:04 | ETA:0:06:29 | Loss:0.02221343273061563 | top1:99.28022003173828
101/643 Data:0.002 | Batch:0.695 | Total:0:01:11 | ETA:0:06:21 | Loss:0.021511859855254983 | top1:99.3019790649414
111/643 Data:0.002 | Batch:0.694 | Total:0:01:18 | ETA:0:06:14 | Loss:0.020993103600434354 | top1:99.30630493164062
121/643 Data:0.001 | Batch:0.702 | Total:0:01:25 | ETA:0:06:06 | Loss:0.02072571254958791 | top1:99.31818389892578
131/643 Data:0.002 | Batch:0.710 | Total:0:01:32 | ETA:0:06:01 | Loss:0.020829524176607606 | top1:99.31297302246094
141/643 Data:0.002 | Batch:0.703 | Total:0:01:39 | ETA:0:05:53 | Loss:0.02176641977345901 | top1:99.28369140625
151/643 Data:0.002 | Batch:0.692 | Total:0:01:46 | ETA:0:05:46 | Loss:0.022440928407630185 | top1:99.264892578125
161/643 Data:0.002 | Batch:0.699 | Total:0:01:53 | ETA:0:05:38 | Loss:0.022607125072907772 | top1:99.2546615600586
171/643 Data:0.002 | Batch:0.695 | Total:0:02:00 | ETA:0:05:31 | Loss:0.02256417559

141/643 Data:0.002 | Batch:0.705 | Total:0:01:40 | ETA:0:05:52 | Loss:0.020448113396988377 | top1:99.27305603027344
151/643 Data:0.002 | Batch:0.699 | Total:0:01:47 | ETA:0:05:46 | Loss:0.02096271549894271 | top1:99.26158142089844
161/643 Data:0.002 | Batch:0.695 | Total:0:01:54 | ETA:0:05:38 | Loss:0.021001990304635086 | top1:99.24845123291016
171/643 Data:0.002 | Batch:0.698 | Total:0:02:01 | ETA:0:05:32 | Loss:0.020990591788969455 | top1:99.24853515625
181/643 Data:0.002 | Batch:0.697 | Total:0:02:08 | ETA:0:05:25 | Loss:0.02081787744562723 | top1:99.25690460205078
191/643 Data:0.002 | Batch:0.703 | Total:0:02:15 | ETA:0:05:18 | Loss:0.021099878600627887 | top1:99.23822021484375
201/643 Data:0.002 | Batch:0.701 | Total:0:02:22 | ETA:0:05:10 | Loss:0.02094387546973998 | top1:99.24378204345703
211/643 Data:0.002 | Batch:0.704 | Total:0:02:29 | ETA:0:05:04 | Loss:0.02115105350184038 | top1:99.22748565673828
221/643 Data:0.002 | Batch:0.695 | Total:0:02:36 | ETA:0:04:56 | Loss:0.0210765